In [2]:
%pip install pip
%pip install bitsandbytes==0.39.0
%pip install torch==2.0.1
%pip install-U git+https://github.com/huggingface/transformers.git@e03a9cc
%pip install-U git+https://github.com/huggingface/peft.git@42a184f
%pip install-U git+https://github.com/huggingface/accelerate.git@c9fbb71
%pip install datasets==2.12.0
%pip install-loralib==0.1.1
%pip install einops==0.6.1

Note: you may need to restart the kernel to use updated packages.


No pyvenv.cfg file


Note: you may need to restart the kernel to use updated packages.


No pyvenv.cfg file


Note: you may need to restart the kernel to use updated packages.


...

No pyvenv.cfg file


Note: you may need to restart the kernel to use updated packages.


No pyvenv.cfg file


Note: you may need to restart the kernel to use updated packages.


No pyvenv.cfg file


In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "abhishek/llama-2-7b-hf-small-shards" #vilsonrodrigues/falcon-7b-instruct-sharded

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)hf-small-shards/resolve/main/config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00010.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00010.bin:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

pytorch_model-00003-of-00010.bin:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

...

(…)hards/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)small-shards/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)ll-shards/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)rds/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:10                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/mapping.py:120 in get_peft_model                    │
│                                                                                                  │
│   117 │   │   return PeftModel(model, peft_config)                                               │
│   118 │   if isinstance(peft_config, PromptLearningConfig):                                      │
│   119 │   │   peft_config = _prepare_prompt_learning_config(peft_config, model_config)           │
│ ❱ 120 │   return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](model, peft_config)     │
│   121                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:689 in __init__                       │
│                                                                                                  │
│    686 │   """                                                                                   │
│    687 │                                                                                         │
│    688 │   def __init__(self, model, peft_config: PeftConfig, adapter_name="default"):           │
│ ❱  689 │   │   super().__init__(model, peft_config, adapter_name)                                │
│    690 │   │   self.base_model_prepare_inputs_for_generation = self.base_model.prepare_inputs_f  │
│    691 │                                                                                         │
│    692 │   def forward(                                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:99 in __init__                        │
│                                                                                                  │
│     96 │   │   self.base_model_torch_dtype = getattr(model, "dtype", None)                       │
│     97 │   │   if not isinstance(peft_config, PromptLearningConfig):                             │
│     98 │   │   │   self.peft_config[adapter_name] = peft_config                                  │
│ ❱   99 │   │   │   self.base_model = PEFT_TYPE_TO_MODEL_MAPPING[peft_config.peft_type](          │
│    100 │   │   │   │   self.base_model, self.peft_config, adapter_name                           │
│    101 │   │   │   )                                                                             │
│    102 │   │   │   self.set_additional_trainable_modules(peft_config, adapter_name)              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:174 in __init__                      │
│                                                                                                  │
│   171 │   │   self.model = model                                                                 │
│   172 │   │   self.forward = self.model.forward                                                  │
│   173 │   │   self.peft_config = config                                                          │
│ ❱ 174 │   │   self.add_adapter(adapter_name, self.peft_config[adapter_name])                     │
│   175 │                                                                                          │
│   176 │   def add_adapter(self, adapter_name, config=None):                                      │
│   177 │   │   if config is not None:                       

# Test original model

In [ ]:
prompt = """
<human>: Sollte
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: warum ist sollte ich programmieren lernen, wenn ich mich mit freunden treffen kann?
<assistant>: Es ist wichtig, programmieren zu lernen, weil es immer wieder neue Herausforderungen auf den Weg kommen.
User 
CPU times: user 10.9 s, sys: 367 ms, total: 11.3 s
Wall time: 17.6 s


# Prep dataset

In [ ]:
data = load_dataset("csv", data_files="dataset.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-398a665bd05b7507/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Input', 'Output'],
        num_rows: 885
    })
})

In [ ]:
data["train"][0]

{'Input': 'Warum ist Programmieren als Hobby so beliebt?',
 'Output': 'Programmieren als Hobby erfreut sich großer Beliebtheit, weil es die kreative Problemlösung fördert und die Möglichkeit bietet, einzigartige Softwareprojekte zu entwickeln.'}

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Input"]}
<assistant>: {data_point["Output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
def generate_prompt(data_point):
  return f"""
### HUMAN:
{data_point["Input"]}

### RESPONSE:
{data_point["Output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/885 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Input', 'Output', 'input_ids', 'attention_mask'],
    num_rows: 885
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.581100
2,2.974400
3,3.528200
4,3.128500
5,3.224400
6,3.169900
7,3.370900
8,2.961900
9,2.776400
10,3.093600


Step,Training Loss
1,3.581100
2,2.974400
3,3.528200
4,3.128500
5,3.224400
6,3.169900
7,3.370900
8,2.961900
9,2.776400
10,3.093600


TrainOutput(global_step=221, training_loss=1.5362878484963292, metrics={'train_runtime': 1246.624, 'train_samples_per_second': 0.71, 'train_steps_per_second': 0.177, 'total_flos': 961992743102976.0, 'train_loss': 1.5362878484963292, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "blockplacer4/Hobby-Ki-V4"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/blockplacer4/Hobby-Ki-V4/commit/80c91a566bc7c668df32ce16b40eb4aced0b125c', commit_message='Upload model', commit_description='', oid='80c91a566bc7c668df32ce16b40eb4aced0b125c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: Was hällst du vom No Nut November?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))